## Computer Vision Algorithm for Tracking Planarian Motion
developed by Hokin Deng xueqiandeng@yahoo.com

# Import Dependencies and Data

Import packages and make sure of the python technicality

In [2]:
import sys
import os
print("Python Version:", sys.version)
print("Python Executable:", sys.executable)
print("Python Path:", sys.path)

Python Version: 3.11.4 (main, Jul  5 2023, 08:41:25) [Clang 14.0.6 ]
Python Executable: /Users/billdeng/anaconda3/envs/unlearning_Version1/bin/python
Python Path: ['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/PycharmProjects/unicellular', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python311.zip', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/lib-dynload', '', '/Users/billdeng/anaconda3/envs/unlearning_Version1/lib/python3.11/site-packages']


In [3]:
import cv2
import math
import copy
import numpy as np
from collections import deque
import numpy as np
import argparse
import imutils

The video data should be named as "sample.avi" and put in the folder as the notebook.

In [4]:
# Open a video file (replace 'sample.avi' with the path)
cap = cv2.VideoCapture('sample.avi')

# Check if the video file was opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

Display the video if want to have a look at it

In [5]:
#cv2.startWindowThread()
# Loop to read and display frames
#while True:
    # Read a frame from the video
#    ret, frame = cap.read()
    # If the video has ended, break out of the loop
#    if not ret:
#        break
    # Display the frame in a window
#    cv2.imshow('Video', frame)
    # Exit the loop if the 'q' key is pressed
#    if cv2.waitKey(25) & 0xFF == ord('q'):
#        break

# Release the video capture object and close the window

## Processing

First, get basic properties about our video

In [6]:
# Get basic video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
video_duration_sec = frame_count / frame_rate if frame_rate > 0 else 0

# Print the video properties
print(f"Frame Count: {frame_count}")
print(f"Frame Width: {frame_width}")
print(f"Frame Height: {frame_height}")
print(f"Frame Rate: {frame_rate} frames per second")
print(f"Video Duration: {video_duration_sec:.2f} seconds")

Frame Count: 1202
Frame Width: 2160
Frame Height: 2160
Frame Rate: 10.0 frames per second
Video Duration: 120.20 seconds


In [7]:
example_ret, example_frame = cap.read()
print( 'ret', type(example_ret))
print( 'frame', type(example_frame))
print('ret', example_ret)
print('frame', example_frame)

ret <class 'bool'>
frame <class 'numpy.ndarray'>
ret True
frame [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]


In [8]:
# frame_number = 100  # The frame you want to access
# Set the video position to the desired frame
# cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)
# Read the frame
# ret, frame = cap.read()
# if ret:
    # Process the frame
#     cv2.imshow('Frame 100', frame)
#    cv2.waitKey(0)  # Wait for a key press to close the image window
# else:
#    print("Error: Unable to read the frame")

Inspect function for a frame

In [9]:
def inspect_frame_from_video(cap_for_here, frame_number):
    """
    Inspects a specific frame in a video.

    :param cap_for_here: video reference
    :param frame_number: The frame number to inspect (1-based index).
    :return: None
    """
    # Open the video file
    if not cap_for_here.isOpened():
        print("Error: Unable to open video file")
        return
    # Check if the frame number is valid
    total_frames = cap_for_here.get(cv2.CAP_PROP_FRAME_COUNT)
    if frame_number < 1 or frame_number > total_frames:
        print(f"Frame number should be between 1 and {int(total_frames)}")
        return
    # Set the video position to the desired frame (0-based index for frame_number)
    cap_for_here.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)
    # Read the frame
    ret_temp, frame_temp = cap_for_here.read()
    if ret_temp:
        # Display the frame
        # cv2.imshow(f'Frame {frame_number}', frame)
        # cv2.waitKey(0)  # Wait for a key press to close the image window
        # cv2.destroyAllWindows()
        frame_h, frame_w = frame_temp.shape[:2]
        print(f"Frame Width: {frame_h}")
        print(f"Frame Height: {frame_w}")
        # Color Channels
        channels = frame_temp.shape[2] if len(frame_temp.shape) == 3 else 1
        print(f"Color Channels: {channels}")
        # Data Type
        data_type = frame_temp.dtype
        print(f"Data Type: {data_type}")
        # Aspect Ratio
        aspect_ratio = frame_w / frame_h
        print(f"Aspect Ratio: {aspect_ratio}")
        # Resolution (assuming a standard display resolution of 96 PPI)
        # Color Space (assuming default BGR)
        color_space = "BGR" if channels == 3 else "Grayscale"
        print(f"Color Space: {color_space}")
        # Histogram for each channel
        if channels > 1:
            for i, col in enumerate(['Blue', 'Green', 'Red']):
                hist = cv2.calcHist([frame_temp], [i], None, [256], [0, 256])
                print(f"Histogram for {col} channel: {np.array(hist).flatten()}")
        else:
            hist = cv2.calcHist([frame_temp], [0], None, [256], [0, 256])
            print(f"Histogram for Grayscale: {np.array(hist).flatten()}")
    else:
        print("Error: Unable to read the frame")

In [10]:
def inspect_given_a_frame(this_frame):
    frame_h, frame_w = this_frame.shape[:2]
    print(f"Frame Width: {frame_h}")
    print(f"Frame Height: {frame_w}")
    channels = this_frame.shape[2] if len(this_frame.shape) == 3 else 1
    print(f"Color Channels: {channels}")
    data_type = this_frame.dtype
    print(f"Data Type: {data_type}")
    aspect_ratio = frame_w / frame_h
    print(f"Aspect Ratio: {aspect_ratio}")
    color_space = "BGR" if channels == 3 else "Grayscale"
    print(f"Color Space: {color_space}")

In [11]:
# Usage example
video_file = 'sample.avi'
frame_to_inspect = 100  # Adjust the frame number as needed
inspect_frame_from_video(cap, frame_to_inspect)

Frame Width: 2160
Frame Height: 2160
Color Channels: 3
Data Type: uint8
Aspect Ratio: 1.0
Color Space: BGR
Histogram for Blue channel: [0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
 0.000000e+00 0.000000e+00 4.000000e+00 1.000000e+01 3.000000e+01
 8.700000e+01 2.260000e+02 2.210000e+02 3.280000e+02 2.530000e+02
 2.220000e+02 2.660000e+02 3.060000e+02 3.730000e+02 3.690000e+02
 4.090000e+02 4.690000e+02 4.750000e+02 5.660000e+02 6.110000e+02
 8.480000e+02 1.009000e+03 8.500000e+02 9.110000e+02 8.800000e+02
 8.680000e+02 9.620000e+02 9.770000e+02 9.740000e+02 9.730000e+02
 1.063000e+03 1.188000e+03 1.234000e+03 1.331000e+03 1.451000e+03
 1.457000e+03 1.428000e+03 1.485000e+03 1.477000e+03 1.456000e+03
 1.4770

In [12]:
ret, frame_for_use = cap.read()
inspect_given_a_frame(frame_for_use)
cap.release()
cv2.destroyAllWindows()

Frame Width: 2160
Frame Height: 2160
Color Channels: 3
Data Type: uint8
Aspect Ratio: 1.0
Color Space: BGR


Background subtraction

In [18]:
i = 0
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('background_remove.avi', fourcc, frame_rate, (frame_width, frame_height), False)
backSub = cv2.createBackgroundSubtractorMOG2()
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    fgMask = backSub.apply(frame)
    out.write(fgMask)
    print(f'\rProgress: {i}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

Progress: 1202

Spatial Smoothing

In [21]:
i = 0
cap = cv2.VideoCapture('background_remove.avi')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('background_remove_spatial_smoothed.avi', fourcc, frame_rate, (frame_width, frame_height), False)
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
    out.write(blurred_frame)
    print(f'\rProgress: {i}', end='')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

Progress: 1202

Temporal Smoothing

In [25]:
cap = cv2.VideoCapture('background_remove_spatial_smooth.avi')
fourcc = cv2.VideoWriter_fourcc(*'XVID') # You can also use 'XVID'
out = cv2.VideoWriter('b_r_s_s_t_s.avi', fourcc, frame_rate, (frame_width, frame_height), False)
buffer_size = 5
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_buffer.append(frame)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)
    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)
    # Write frame to video
    out.write(temp_smoothed)
    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

OpenCV: Couldn't read video stream from file "background_remove_spatial_smooth.avi"
[ERROR:0@834.234] global cap.cpp:166 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.1) /Users/runner/work/opencv-python/opencv-python/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): background_remove_spatial_smooth.avi in function 'icvExtractPattern'




do everything all at once

In [27]:
# Initialize video capture
cap = cv2.VideoCapture('sample.avi')
# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You can also use 'XVID'
out = cv2.VideoWriter('pre_dense.mp4', fourcc, frame_rate, (frame_width, frame_height), False)
# Background subtractor
backSub = cv2.createBackgroundSubtractorMOG2()
# Buffer for temporal smoothing
buffer_size = 5
frame_buffer = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Background subtraction
    fgMask = backSub.apply(frame)
    # Spatial smoothing (Gaussian blur)
    blurred = cv2.GaussianBlur(fgMask, (5, 5), 0)
    # Add frame to buffer for temporal smoothing
    frame_buffer.append(blurred)
    if len(frame_buffer) > buffer_size:
        frame_buffer.pop(0)
    # Temporal smoothing (average of frames in buffer)
    temp_smoothed = np.mean(frame_buffer, axis=0).astype(np.uint8)
    # Write frame to video
    out.write(temp_smoothed)
    # Display result
    # cv2.imshow('Frame', frame)
    # cv2.imshow('FG Mask', fgMask)
    # cv2.imshow('Blurred', blurred)
    # cv2.imshow('Temporally Smoothed', temp_smoothed)
    # Break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

dense opti flow
this takes a very very long time

In [29]:
cap = cv2.VideoCapture("pre_dense.mp4")
first_ret, first_frame = cap.read()
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('dense_opti_flow.mp4', fourcc, frame_rate, (frame_width, frame_height), False)
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)
mask[..., 1] = 255
i = 0
while True:
  i += 1
  ret, frame = cap.read()
  if not ret:
    break
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  flow = cv2.calcOpticalFlowFarneback(prev_gray, gray,
									None,
									0.5, 3, 15, 3, 5, 1.2, 0)
  magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
  mask[..., 0] = angle * 180 / np.pi / 2
  mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
  rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
  out.write(rgb)
  print(f'\rProgress: {i}', end='')
  prev_gray = gray
cap.release()
out.release()
cv2.destroyAllWindows()

Progress: 1201

In [30]:
cap = cv2.VideoCapture("pre_dense.mp4")
if not cap.isOpened():
    print("Error opening video file")
    exit()
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Try 'XVID' if 'mp4v' does not work
out = cv2.VideoWriter('dense_opti_flow_v2.mp4', fourcc, frame_rate, (frame_width, frame_height), True)
ret, first_frame = cap.read()
if not ret:
    print("Error reading first frame")
    cap.release()
    exit()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
mask = np.zeros_like(first_frame)
mask[..., 1] = 255
i = 0
while True:
    i += 1
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    mask[..., 0] = angle * 180 / np.pi / 2
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    out.write(rgb)
    print(f'\rProgress: {i}', end='')
    prev_gray = gray
cap.release()
out.release()
cv2.destroyAllWindows()

Progress: 1201

Define a function for video to image and use it

In [ ]:
# Define a function to extract and save frames from a video file
current_location = os.getcwd();
output_folder = current_location + '/raw_images'
video_path = current_location + '/sample.avi'

def video2image(video_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the video
    video = cv2.VideoCapture(video_path)

    # Initialize frame count
    count = 0

    # Iterate through video frames
    while True:
        # Read a frame
        success, frame = video.read()
        # Break if no frame is read (end of video)
        if not success:
            break
        # Save the frame as an image
        cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), frame)
        # Increment frame count
        count += 1

    # Release the video object
    video.release()

    return count

Save the video into images

In [ ]:
frame_count = video2image(video_path, output_folder)

Design a crop function that remove the dish

In [ ]:
def crop_circle(image_path):

    # Read the image
    image = cv2.imread(image_path)

    # Create a mask with the same dimensions as the image
    mask = np.zeros_like(image)
    rows, cols, _ = mask.shape

    # Compute the center and radius of the circle
    center = (cols // 2, rows // 2)
    radius = min(center[0], center[1], rows - center[1], cols - center[0])

    # Draw the circular mask
    cv2.circle(mask, center, radius, (255, 255, 255), -1)

    # Apply the mask
    circular_image = cv2.bitwise_and(image, mask)

    # Optionally, you can remove the black background
    masked_data = cv2.cvtColor(circular_image, cv2.COLOR_BGR2BGRA)
    masked_data[mask == 0] = [0, 0, 0, 0]

    # Save or display the result
    cv2.imwrite('circular_image.png', masked_data)
    # cv2.imshow('Circular Image', masked_data)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Use the function
crop_circle('path_to_your_image.jpg')